## Question 1
Run the command "spark.version" What version number is output?

In [2]:
# Import libraries
import pandas as pd
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

# Ensure that pyspark is available and that the environment variables are set
findspark.init()

# Create a SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

spark.version

'3.3.2'

## Question 2
Read it with Spark using the same schema as we did in the lessons.
We will use this dataset for all the remaining questions.
Repartition it to 12 partitions and save it to parquet.
What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [ ]:
# Download data
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz

In [16]:
# Check the column names and default types - IMPORTANT
df = spark.read.option('header', "True").csv('fhvhv_tripdata_2021-06.csv.gz')
df.limit(5).dtypes

[('dispatching_base_num', 'string'),
 ('pickup_datetime', 'string'),
 ('dropoff_datetime', 'string'),
 ('PULocationID', 'string'),
 ('DOLocationID', 'string'),
 ('SR_Flag', 'string'),
 ('Affiliated_base_number', 'string')]

In [17]:
# Create an appropriate schema for the dataframe
schema = types.StructType([
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.TimestampType(),True),
    types.StructField('dropoff_datetime',types.TimestampType(),True),
    types.StructField('PULocationID',types.IntegerType(),True),
    types.StructField('DOLocationID',types.IntegerType(),True),
    types.StructField('SR_Flag',types.StringType(),True),
    types.StructField('Affiliated_base_number',types.StringType(),True),
])

# Read CSV data and write it in 12 partitions in PARQUET format
df = spark.read \
    .option("header", "True") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv.gz')
df.repartition(12).write.parquet('fhvhv/2021/06/') 
# ANSWER: 24 MB

## Question 3
Count records

How many taxi trips were there on June 15?

Consider only trips that started on June 15.

In [21]:
# Create date column
df = df.withColumn('pickup_date', F.to_date(df.pickup_datetime))
df = df.withColumn('dropoff_date', F.to_date(df.dropoff_datetime))
df.filter("pickup_date = '2021-06-15'").count()
# ANSWER: 452470
# df.filter((F.col("pickup_date") == '2021-06-15') & (F.col("dropoff_date") == '2021-06-15')).count()

452470

## Question 4
Longest trip for each day

Now calculate the duration for each trip.
How long was the longest trip in Hours?

In [24]:
from pyspark.sql.functions import unix_timestamp, col
# Convert the pickup and dropoff timestamps to Unix timestamps:
df = df.withColumn('pickup_timestamp', unix_timestamp(col('pickup_datetime')))
df = df.withColumn('dropoff_timestamp', unix_timestamp(col('dropoff_datetime')))
# Estimate duration of each trip in seconds
df = df.withColumn('duration', (col('dropoff_timestamp') - col('pickup_timestamp')) / 3600.0)
# Estimate the maximum duration for each date
df.groupBy('pickup_date') \
    .agg({'duration': 'max'}) \
    .orderBy('max(duration)', ascending=False) \
    .show()


+-----------+------------------+
|pickup_date|     max(duration)|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
| 2021-06-24|13.909722222222221|
| 2021-06-04|             11.67|
| 2021-06-20|10.984444444444444|
| 2021-06-01|           10.2675|
| 2021-06-28| 9.966388888888888|
| 2021-06-18| 9.624444444444444|
| 2021-06-08| 9.480277777777777|
| 2021-06-11| 9.471666666666666|
| 2021-06-15| 9.402222222222223|
| 2021-06-03| 9.365833333333333|
| 2021-06-19| 9.106944444444444|
| 2021-06-30| 9.056111111111111|
| 2021-06-09| 9.030277777777778|
| 2021-06-17| 8.774166666666666|
| 2021-06-29| 8.571666666666667|
+-----------+------------------+
only showing top 20 rows



In [29]:
# Another way
df.registerTempTable('fhvhv_2021_06')
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM fhvhv_2021_06
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
| 2021-06-24|13.909722222222221|
| 2021-06-04|             11.67|
| 2021-06-20|10.984444444444444|
| 2021-06-01|           10.2675|
| 2021-06-28| 9.966388888888888|
+-----------+------------------+



## Question 5
Spark’s User Interface which shows application's dashboard runs on which local port?
ANSWER: 4040

## Question 6
Load the zone lookup data into a temp view in Spark
Zone Data

Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

In [37]:
# Read zones dataframe
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')
df_zones.registerTempTable('df_zones')
# Count trips per each pick up location
spark.sql("""
SELECT
    pul.Zone,
    count(1) as trips
FROM fhvhv_2021_06 AS fhv 
    LEFT JOIN df_zones AS pul ON fhv.PULocationID = pul.LocationID
GROUP BY 1
ORDER BY 2 DESC
""").show()
# ANSWER: Crown Heights North

+--------------------+------+
|                Zone| trips|
+--------------------+------+
| Crown Heights North|231279|
|        East Village|221244|
|         JFK Airport|188867|
|      Bushwick South|187929|
|       East New York|186780|
|TriBeCa/Civic Center|164344|
|   LaGuardia Airport|161596|
|            Union Sq|158937|
|        West Village|154698|
|             Astoria|152493|
|     Lower East Side|151020|
|        East Chelsea|147673|
|Central Harlem North|146402|
|Williamsburg (Nor...|143683|
|          Park Slope|143594|
|  Stuyvesant Heights|141427|
|        Clinton East|139611|
|West Chelsea/Huds...|139431|
|             Bedford|138428|
|         Murray Hill|137879|
+--------------------+------+
only showing top 20 rows



In [38]:
spark.stop()